# Envio múltiplo

**Envio múltiplo** é uma característia fundamental de Julia, que exploraremos neste bloco de notas.

Ajuda a tornar o softwerw mais rápido.Ele também torna o softwerw extensível, programável e totalmente dovertido de usar.

Podendo ser um grande avanço para a computação paralela.

1. Algarismos romanos
2. Funções
3. Computação paralela

## 1. Algarismos romanos (para diversão)

Vamos definir uma **nova estrutura** que representa um algarismos romano.Para simplificar a codificação, lidaremos apenas com números entre 0 e 9.

**Exercício**:
Estenda para números maiores.(Lembre-se de que os números romanos são um sistema de base 10!)

In [2]:
struct Roman
    n::Int
end

Base.show(io::IO, r::Roman) = print(io, 'ⅰ' + (r.n - 1) % 10)  # nice display; 'ⅰ' is a Unicode Roman numeral

Podemos criar um objeto deste tipo da seguinte maneira :

In [3]:
Roman(5)

ⅴ

In [4]:
typeof.([5 5.0 Roman(5) "Five" '5'  5//1])

1×6 Matrix{DataType}:
 Int64  Float64  Roman  String  Char  Rational{Int64}

In [5]:
x = [7 1 2 5 8 9]
Roman.(x)   # equivalente a map (Roman, x) ou [Roman (w) para w em x]

1×6 Matrix{Roman}:
 ⅶ  ⅰ  ⅱ  ⅴ  ⅷ  ⅸ

Seria bom poder adicionar algarismos romanos como números normais:

In [ ]:
Roman(4) + Roman(5)

Mas Julia não sabe fazer isso. Vamos ensiná-la `importando` a função `+`, que nos permite estender sua definição:

In [ ]:
import Base: +, *

+(a::Roman, b::Roman) = Roman(a.n + b.n)

In [ ]:
Roman(4) + Roman(5)

Isso **adiciona um novo método** à função `+`:

In [ ]:
methods(+)

In [ ]:
import Base.*
*(i::Roman, j::Roman) = Roman(i.n * j.n)                     # Multiplique como um romano

In [ ]:
Roman(3) * Roman(2)

In [ ]:
Roman.(1:3) .* [Roman(1) Roman(2) Roman(3)]

Mas 

In [ ]:
Roman(3) * 2

In [ ]:
#  Mytimes complicado para decidir o que fazer com base no tipo
# não sugerido, melhor maneira em breve

function mytimes(i,j)
  if isa(i,Roman) & isa(j,Number)
        return  fill(1, i.n, j)   # i by j matrix with ones
    elseif    isa(i,Number) & isa(j,Roman) 
        return "😄"^(i*j.n)   #  i * j happy faces
    else
        return("I Don't know")
    end
end

In [ ]:
mytimes(4,Roman(3)) # Twelve happys

In [ ]:
mytimes(Roman(4),3) # 4x3 matriz com uns

A coisa mais simples a fazer é definir explicitamente a multiplicação de um `Romano`por um número. Podemos fazer como acharmos adequado:

In [ ]:
*(i::Number, j::Roman) = "😄"^(i*j.n)        #  i * j caras felizes

*(i::Roman, j::Number) = fill(1, i.n, j)       # matriz i por j

In [ ]:
3 * Roman(3) # Nove felizes

In [ ]:
Roman(3) * 5  # Matriz três por cinco de uns

In [ ]:
t(x::Roman,y::Roman) = x.n * y.n

In [ ]:
t(Roman(5),Roman(4))

In [ ]:
# Observe como o montador é apertado!
@code_native t(Roman(2),Roman(4))

# Funções

In [ ]:
import Base: *, +, ^

In [ ]:
*(α::Number,   g::Function) = x -> α * g(x)   # Função de tempos escalares

*(f::Function, λ::Number)   = x -> f(λ * x)   # Escale o argumento

*(f::Function, g::Function) = x -> f(g(x))    # Function composition  -- abuse of notation!  use \circ in Julia 0.6

^(f::Function, n::Integer) = n == 1 ? f : f*f^(n-1) # Um algoritmo de exponenciação ingênua por multiplicação recursiva

In [ ]:
+(f::Function, g::Function) = x -> f(x) + g(x)

Por exemplo, a função exponencial é definida como

$$\exp(x) = \sum_{n=0}^\infty \frac{1}{n!} x^n.$$

Podemos pensar nisso apenas em termos de funções:

$$\exp = \sum_{n=0}^\infty \frac{1}{n!} \mathrm{pow}_n,$$

where $\mathrm{pow}_n(x) = x^n$.

(começa a confundir o simbólico com o numérico!)

In [ ]:
pow(n) = x -> x^n

myexp = sum(1/factorial(big(n)) * pow(n) for n in 0:100)   # taylor series not efficient!

In [ ]:
[myexp(1); exp(1); exp(big(1))]

In [ ]:
f = x -> x^2
f(10)

In [ ]:
g = 3f
g(10)

In [ ]:
(f^2)(10)  # uma vez que definimos multiplicação de funções como composição

In [ ]:
using Plots;
gr()

In [ ]:
x = pi*(0:0.001:4)

plot(x, sin.(x),    c="black", label="Fun")
plot!(x, (12*sin).(x),    c="green", label="Num * Fun")
plot!(x, (sin*12).(x),    c="red", alpha=0.9, label="Fun * Num")
plot!(x, (5*sin*exp).(x), c="blue", alpha=0.2, label="Num * Fun * Fun")

In [ ]:
plot([12*sin, sin*12, 5*sin*exp], 0:.01:4π, α=[1 .9 .2], c=[:green :red :blue])

<img src="https://lh4.googleusercontent.com/--z5eKJbB7sg/UffjL1iAd4I/AAAAAAAABOc/S_wDVyDOBfQ/gauss.jpg">

### "Sin^2 phi é odioso para mim, embora Laplace tenha feito uso dele; deve-se temer que o sin^ 2 phi possa se tornar ambíguo, o que talvez nunca ocorresse, ou no máximo muito raramente quando se fala de sin( phi ^ 2), bem então, vamos escrever (sin phi)^ 2, mas não sin ^ 2 phi, que por analogia deveria significar sin (sin phi). " - Gauss

In [ ]:
x=(0:.01:2) * pi;

plot(x, (sin^2).(x), c="blue")     # Quadrar simplesmente funciona, y = sin (sin (x)), Gauss ficaria satisfeito!
plot!(x, sin.(x).^2,  c="red")         

# Exercício

In [ ]:
h(a, b::Any) = "fallback"
h(a::Number, b::Number) = "a and b are both numbers"
h(a::Number, b) = "a is a number"
h(a, b::Number) = "b is a number"
h(a::Integer, b::Integer) = "a and b are both integers"

In [ ]:
# Experimente brincar com h